In [1]:
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
from torch import tensor

from torch.autograd import grad
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

from pdefind import *
from PDE_Equation import pde_matrix_mul, sparse_coeff, scale_xi_threshold, pde_Recover



ImportError: cannot import name 'scale_xi_threshold' from 'PDE_Equation' (D:\Thesis\Neural Network\Project\Model Identification\PDE_Equation.py)

In [ ]:
polynm = ['1', 'u', 'uˆ2']
spa_der = ['1', 'u_{x}', 'u_{xx}']
library_coeffs = pde_matrix_mul(polynm, spa_der)
print('library_coeffs:', library_coeffs)

tot_items = len(library_coeffs)
print('tot_items:', tot_items)

In [ ]:
# Data Setup
data = sio.loadmat(os.path.join(os.getcwd(), "data", "burgers.mat"))
u = data["usol"]
x = data["x"][0]
t = np.squeeze(data["t"], axis=1)

print("u shape", u.shape)
print("x shape", x.shape)
print("t shape", t.shape)

im = plt.imshow(u.real, cmap="seismic", aspect=0.4)
plt.colorbar(im)
plt.title("Burgers Equation")
plt.xlabel("t")
plt.ylabel("x")

xpos  = np.arange(len(t), step=25)
ypos  = np.where(x%4==0)[0].tolist()
ytick = x[ypos].tolist()
ypos  += [len(x)-1]
ytick += [8.0]

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)
plt.gcf().set_size_inches(6,5)
plt.show()

In [ ]:
# Add Noise
np.random.seed(0)
noise_level = 0.1
u = u.real + noise_level*np.std(u.real)*np.random.randn(u.shape[0],u.shape[1])

im = plt.imshow(u.real, cmap="seismic", aspect=0.4)
plt.colorbar(im)
plt.title("Noisy Data")
plt.xlabel("t")
plt.ylabel("x")

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)
plt.gcf().set_size_inches(6,5)
plt.show()


In [ ]:
# Prepare Training Data
xx, tt = np.meshgrid(x,t)
X = np.vstack([xx.ravel(), tt.ravel()]).T
print("X shape", X.shape)
print(X)

In [ ]:
y = np.zeros((u.size, 1), dtype=np.float)
for i,_x in enumerate(u.real.T):
    y[i*len(x):(i+1)*len(x)] = _x.reshape(len(x),1)
    
print("y shape", y.shape)

plt.plot(y[:256], label="t=0")
plt.plot(y[50*256:51*256], label="t=50")
plt.plot(y[100*256:101*256], label="t=100")
plt.xlabel("x")
plt.ylabel("u")
plt.xticks(ypos, ytick)
plt.legend()
plt.show()

In [ ]:
idxs = np.random.choice(y.size, 1000, replace=False)

X_train = torch.tensor(X[idxs], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y[idxs], dtype=torch.float32)
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X shape", X.shape)
print("y shape", y.shape)

plt.subplot(1,2,1)
im = plt.imshow(u, cmap="seismic", aspect=0.4)
plt.colorbar(im)
plt.xlabel("t")
plt.ylabel("x")
plt.title("Noisy")

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)

usamp = np.full(y.shape,np.nan)
usamp[idxs] = y[idxs]
usamp = usamp.reshape(u.T.shape).T

plt.subplot(1,2,2)
im = plt.imshow(usamp, cmap="seismic", aspect=.4)
plt.colorbar(im)
plt.xlabel("t")
plt.ylabel("x")
plt.title("Sampled")

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)

plt.gcf().set_size_inches(10,4)
plt.show()

In [ ]:
# Setup the network
net = PINN(sizes=[2,20,15,10,5,1], activation=torch.nn.Tanh())
print(net)

In [ ]:
# Building Library
def build_library(uhat, data, poly_order, deriv_order):
    # build polynomials
    poly = torch.ones_like(uhat)
    
    # concatinate different orders
    for o in np.arange(1, poly_order+1):
        poly_o = poly[:,o-1:o]*uhat
        #print('poly_o:',poly_o)
        poly = torch.cat((poly, poly_o), dim=1)
        
    # build derivatives
    # returns gradient of uhat w.r.t. data (id0=spatial, id1=temporal)
    du = grad(outputs=uhat, inputs=data, 
              grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
    
    # time derivative
    dudt = du[:,1:2]
    
    # spatial derivatives
    dudx = torch.cat((torch.ones_like(dudt), du[:,0:1]), dim=1)
    
    # concatinate different orders
    for o in np.arange(1, deriv_order):
        du = grad(outputs=dudx[:,o:o+1], inputs=data, 
                  grad_outputs=torch.ones_like(uhat), create_graph=True)[0]
        dudx = torch.cat((dudx, du[:,0:1]), dim=1)
    
    # build all possible combinations of poly and dudx vectors
    theta = None
    for i in range(poly_order+1):
        for j in range(deriv_order+1):
            comb = poly[:,i:i+1] * dudx[:,j:j+1]
            
            if theta is None:
                theta = comb
            else:
                theta = torch.cat((theta, comb), dim=1)
                
    return dudt, theta

    

In [ ]:
epochs = 4000
#xi = nn.Parameter(torch.randn((1, 1), requires_grad=True, device="cpu", dtype=torch.float32))
xi = nn.Parameter(torch.randn((tot_items, 1), requires_grad=True, device="cpu", dtype=torch.float32))
#xi = torch.tensor([[0.1], [-1]])
#print(xi)
#params = [{'params': net.parameters(), 'lr': 1e-3}]
params = [{'params': net.parameters(), 'lr': 1e-3}, {'params': xi, 'lr': 2e-2}]

optimizer = Adam(params)
scheduler = ExponentialLR(optimizer, .9998)

In [ ]:
def PI_NeuralNet(uhat, X_train, mask, poly_order, deriv_order):
    lamb   = 0
    tolerance = 10**-6
    mask = torch.ones(tot_items, 1)
    print('xi', xi)
    print('mask:', mask.shape)
    
    dudt, theta = build_library(uhat, X_train, poly_order=2, deriv_order=2)

    dudt_norm = torch.norm(dudt, dim=0)
    print('dudt_norm:', dudt_norm.shape)
    
   
    theta_scaling = (torch.norm(theta, dim=0))
    #print('theta_scaling:', theta_scaling.shape)
    scaling_theta = torch.unsqueeze(theta_scaling, dim = 1)
    theta_norm = torch.unsqueeze(theta_scaling, dim = 1) 
    print('theta_norm:', theta_norm.shape)
    #print('xi:', xi.shape)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        uhat = net(X_train)
    
        if epoch == 1000:
            lamb = 1
               
        dudt, theta = build_library(uhat, X_train, poly_order=2, deriv_order=2)
        #print('dudt:', dudt.shape)
        #theta = theta[:, [2,4]].clone()
        l_u   = nn.MSELoss()(uhat, y_train)
        l_reg = lamb * torch.mean((dudt - theta @ xi)**2)

        loss = l_u + l_reg
        #print('loss', loss)
    

        xi_normalized = xi * (theta_norm / dudt_norm) 
        #print('xi_normalized:', xi_normalized.shape)


        #gradient_losses = torch.max(torch.abs(grad(outputs=loss, inputs=xi, 
              #grad_outputs=torch.ones_like(loss), create_graph=True)[0]))
        
        gradient_losses = torch.max(torch.abs(grad(outputs=loss, inputs=xi, 
              grad_outputs=torch.ones_like(loss), create_graph=True)[0]) / (theta_norm / dudt_norm))
        
        
        gradient_loss = torch.max(gradient_losses)
        
        #gradient_losses = torch.abs(grad(outputs=loss, inputs=xi, create_graph=True)[0])
        #print('gradient_losses:', gradient_losses)

        loss.backward()
        optimizer.step()
    
        #print("epoch {}/{}, loss={:.10f}".format(epoch+1, epochs, loss.item()), end="\r")
        
        if epoch % 1500 == 0:
            #if epoch % 500 == 0:
            print('gradient_loss:', gradient_loss)
            print('loss:', loss)
            #if (gradient_losses) < (tolerance):
            if gradient_loss < tolerance:
                print('Optimizer converged.')
                #break
    #print('xi', xi)    
    #print('xi_normalized:', xi_normalized)
    xi_list = sparse_coeff(mask, xi.detach().numpy())
    #print('xi_list:', xi_list)
    xi_normalized = sparse_coeff(mask, xi_normalized.detach().numpy())
    #print('xi_normalized:', xi_normalized)
    print('Finished')
            
            
    return xi_list, xi_normalized      



In [ ]:
#mask = torch.ones(9, 1)
mask = torch.ones(tot_items, 1)
#xi =torch.randn((tot_items, 1), dtype=torch.float32, requires_grad = True)
uhat = net(X_train)
xi_list, xi_normalized= PI_NeuralNet(uhat, X_train, mask, poly_order=2, deriv_order=2)



In [ ]:
def model_Identification(data, uhat, mask, tot_items, poly_order, deriv_order):
    #mask = torch.ones(9, 1)
    mask = torch.ones(tot_items, 1)
    xi_list, xi_normalized = PI_NeuralNet(uhat, X_train, mask, poly_order, deriv_order)
    sparsity = scale_xi_threshold( xi_normalized, mode='auto')
    #print('sparsity:', sparsity)
    
    mask[~np.transpose(np.squeeze(np.array(sparsity)))] = 0
    xi_thresholded = np.expand_dims(xi_list[sparsity], axis=1) 
    
    # Printing current sparse vector to see progress
    print('Coefficient xi:')
    print(sparse_coeff(sparsity, xi_thresholded))
    
    # Now thats it's converged, fit again but without the L1 penalty
    print('Now running for the final time...')
    
    xi, _ = PI_NeuralNet(uhat, X_train, mask, poly_order, deriv_order)
    
    uhat = net(torch.FloatTensor(data))
    xi_list = sparse_coeff(sparsity, xi_thresholded)
    #xi_list = xi
    print('Finished')
    
    return xi_list, uhat

In [ ]:
xi_list, uhat = model_Identification(X, uhat, mask, tot_items, poly_order=2, deriv_order=2)

In [ ]:
print(uhat.shape)
pde_Recover(xi_list, library_coeffs, equation_form='u_t')

In [ ]:
uhat = net(torch.FloatTensor(X))
print("MSE loss", nn.MSELoss()(uhat, torch.FloatTensor(y)).item())

uhat = uhat.data.reshape(u.T.shape)

plt.subplot(1,3,1)
im = plt.imshow(data["usol"].real, cmap="seismic", aspect=0.4)
plt.colorbar(im)
plt.xlabel("t")
plt.ylabel("x")
plt.title("Original")

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)

plt.subplot(1,3,2)
im = plt.imshow(u, cmap="seismic", aspect=0.4)
plt.colorbar(im)
plt.xlabel("t")
plt.ylabel("x")
plt.title("Noisy")

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)

plt.subplot(1,3,3)
im = plt.imshow(uhat.numpy().T, cmap="seismic", aspect=.4)
plt.colorbar(im)
plt.xlabel("t")
plt.ylabel("x")
plt.title("Reconstructed")

plt.xticks(xpos, t[xpos])
plt.yticks(ypos, ytick)

plt.gcf().set_size_inches(18,4)
plt.show()

In [ ]:
plt.imshow(np.abs(uhat.numpy().T.data - data["usol"].real), cmap="seismic", aspect=.4)
plt.colorbar()
plt.show()

In [ ]:
# Denoising with coefficient
net = PINN(sizes=[2,20,15,10,5,1], activation=torch.nn.Tanh())
print(net)